[View in Colaboratory](https://colab.research.google.com/github/Joovvhan/ColabTest/blob/master/Transfer_Learning_Model_Comparison_Local_DenseNet201.ipynb)

In [0]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [0]:
import os
import matplotlib.pyplot as plt
import numpy as np
import scipy.io.wavfile as wf

In [0]:
# os.listdir('gdrive/My Drive/Colab')

In [0]:
# dataPath = 'gdrive/My Drive/Colab/Data'

dataPath = '''D:/0_Joowhan's Paper/Synthesized/Total'''

"""
folders = os.listdir(dataPath)

for folder in folders:
  files = os.listdir(dataPath + '/' + folder)
  print(len(files))
  print(files[-1])
"""

folderF1 = dataPath + '/' + 'A3F1P3'  
folderF5 = dataPath + '/' + 'A3F5P3'
filesF1 = os.listdir(folderF1)
filesF5 = os.listdir(folderF5)

folderA5 = dataPath + '/' + 'A5F3P3'
filesA5 = os.listdir(folderA5)

In [0]:
filesF1[0]

'A2.501088_F0.064891_P2.393651_H368.201761.wav'

In [0]:
fs, dataInt16 = wf.read(folderF1 + '/' + filesF1[0])
dataFloat = dataInt16 / (2 ** 15)

fs, dataInt16 = wf.read(folderF5 + '/' + filesF5[0])
dataFloat = dataInt16 / (2 ** 15)

In [0]:
nsc = 1470
nov = nsc/2
nff = nsc 

In [0]:
Pxx, freqs, bins, _ = plt.specgram(dataFloat, NFFT=nff, Fs=fs, noverlap=nov, \
                                   window=np.hamming(nsc), cmap='viridis')
plt.close()
img = Pxx[0:224, :]

plt.imshow(10*np.log10(img), cmap='viridis')
plt.close()

In [0]:
import time

In [0]:
from tqdm import tqdm, trange

In [0]:
imgSize = 224
imgsF1 = np.zeros([len(filesF1), imgSize, imgSize])

for i in trange(len(filesF1)):
    fs, dataInt16 = wf.read(folderF1 + '/' + filesF1[i])
    dataFloat = dataInt16 / (2 ** 15)
    Pxx, _, _, _ = plt.specgram(dataFloat, NFFT=nff, Fs=fs, noverlap=nov, \
                                       window=np.hamming(nsc), cmap='viridis')
    plt.close()
    imgsF1[i, :, :] = 10 * np.log10(Pxx[0:224, :])

100%|█████████████████████████████████████████████████████████████████| 1000/1000 [00:27<00:00, 36.27it/s]


In [0]:
imgsF5 = np.zeros([len(filesF5), imgSize, imgSize])

for i in trange(len(filesF5)):
    fs, dataInt16 = wf.read(folderF5 + '/' + filesF5[i])
    dataFloat = dataInt16 / (2 ** 15)
    Pxx, _, _, _ = plt.specgram(dataFloat, NFFT=nff, Fs=fs, noverlap=nov, \
                                       window=np.hamming(nsc), cmap='viridis')
    plt.close()
    imgsF5[i, :, :] = 10 * np.log10(Pxx[0:224, :])

100%|█████████████████████████████████████████████████████████████████| 1000/1000 [00:28<00:00, 37.04it/s]


In [0]:
trainIdxF1 = np.random.choice(len(imgsF1) - 1, int(len(imgsF1) * 0.8), replace=False)
testIdxF1 = list(set(range(0, len(imgsF1))) - set(trainIdxF1))

trainImgsF1 = imgsF1[trainIdxF1, :, :]
testImgsF1 = imgsF1[testIdxF1, :, :]

trainIdxF5 = np.random.choice(len(imgsF5) - 1, int(len(imgsF5) * 0.8), replace=False)
testIdxF5 = list(set(range(0, len(imgsF5))) - set(trainIdxF5))

trainImgsF5 = imgsF5[trainIdxF5, :, :]
testImgsF5 = imgsF5[testIdxF5, :, :]

In [0]:
trainImgs = np.vstack([trainImgsF1, trainImgsF5])
testImgs = np.vstack([testImgsF1, testImgsF5])

In [0]:
trainMean = np.mean(trainImgs)
trainStd = np.std(trainImgs)

In [0]:
trainImgsNorm = (trainImgs - trainMean) / trainStd
testImgsNorm = (testImgs - trainMean) / trainStd

trainImgsNorm = trainImgsNorm.reshape(list(trainImgsNorm.shape) + [1])
testImgsNorm = testImgsNorm.reshape(list(testImgsNorm.shape) + [1])

In [0]:
X_train = np.stack([trainImgsNorm[:, :, :, 0], trainImgsNorm[:, :, :, 0], trainImgsNorm[:, :, :, 0]], axis = -1)
X_test = np.stack([testImgsNorm[:, :, :, 0], testImgsNorm[:, :, :, 0], testImgsNorm[:, :, :, 0]], axis = -1)

In [0]:
trainLabelF1 = np.stack((np.ones( int(len(imgsF1) * 0.8)), np.zeros( int(len(imgsF1) * 0.8))), axis = -1)
testLabelF1 = np.stack((np.ones( int(len(imgsF1) * 0.2)), np.zeros( int(len(imgsF1) * 0.2))), axis = -1)

trainLabelF5 = np.stack((np.zeros( int(len(imgsF5) * 0.8)), np.ones( int(len(imgsF5) * 0.8))), axis = -1)
testLabelF5 = np.stack((np.zeros( int(len(imgsF5) * 0.2)), np.ones( int(len(imgsF5) * 0.2))), axis = -1)

Y_train = np.vstack((trainLabelF1, trainLabelF5))
Y_test = np.vstack((testLabelF1, testLabelF5))

In [0]:
from keras.preprocessing import image
from keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.models import Sequential

Using TensorFlow backend.


In [0]:
from keras.applications.densenet import DenseNet201
from keras import backend as K

# for i in range(1000):

input_tensor = Input(shape=(224, 224, 3))

modelDenseNet201WoTop = None
modelDenseNet201 = None

modelDenseNet201WoTop = DenseNet201(input_tensor=input_tensor, weights='imagenet', include_top=False)

# modelDenseNet201WoTop.summary()

modelDenseNet201 = Sequential()

modelDenseNet201.add(modelDenseNet201WoTop)

modelDenseNet201.add(GlobalAveragePooling2D())

modelDenseNet201.add(Dense(2, activation='relu'))

modelDenseNet201.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

modelDenseNet201.fit(X_train, Y_train,
          batch_size=1, epochs=1, verbose=1,
          validation_data=(X_test, Y_test))

modelDenseNet201.summary()

K.clear_session()

Train on 1600 samples, validate on 400 samples
Epoch 1/1
 757/1600 [=============>................] - ETA: 3:13 - loss: 8.0592 - acc: 0.5000

In [42]:
input_tensor = Input(shape=(224, 224, 3))

modelDenseNet201WoTop = None
modelDenseNet201 = None

modelDenseNet201 = DenseNet201(input_tensor=input_tensor, weights='imagenet')
modelDenseNet201.summary()

82526208/82524592 [==============================] - 18s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
___________________________________________

In [22]:
from keras.applications.mobilenet import MobileNet
from keras import backend as K

# for i in range(1000):

input_tensor = Input(shape=(224, 224, 3))

modelMobileNetWoTop = None
modelMobileNet = None

modelMobileNetWoTop = MobileNet(input_tensor=input_tensor, weights='imagenet', include_top=False)

modelMobileNetWoTop.summary()

modelMobileNet = Sequential()

modelMobileNet.add(GlobalAveragePooling2D())

modelMobileNet.add(Reshape([None, 1, 1, 1024]))

modelMobileNet.add(Dropout(0.5))

modelMobileNet.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

modelMobileNet.fit(X_train, Y_train,
          batch_size=10, epochs=200, verbose=2,
          validation_data=(X_test, Y_test))


global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 1, 1, 1024)        0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 1, 1024)        0         
_________________________________________________________________
conv_preds (Conv2D)          (None, 1, 1, 1000)        1025000   
_________________________________________________________________
act_softmax (Activation)     (None, 1, 1, 1000)        0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 1000)              0    

K.clear_session()

# modelXceptionWoTop.summary()

# modelXception = Xception(input_tensor=input_tensor, weights='imagenet')

# modelXception.summary()

c:\users\주환\appdata\local\conda\conda\envs\paper\lib\site-packages\keras_applications\mobilenet.py:206: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


17227776/17225924 [==============================] - 5s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_b

In [25]:
input_tensor = Input(shape=(224, 224, 3))

modelMobileNet = MobileNet(input_tensor=input_tensor, weights='imagenet')

modelMobileNet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

AttributeError: ignored

In [40]:
lay = modelMobileNet.get_layer('global_average_pooling2d_2')
lay.output.shape

lay = modelMobileNet.get_layer('conv_preds')
lay.output.shape

TensorShape([Dimension(None), Dimension(1), Dimension(1), Dimension(1000)])

In [0]:
from keras.applications.xception import Xception
from keras import backend as K

for i in range(1000):

    input_tensor = Input(shape=(224, 224, 3))

    modelXceptionWoTop = None
    modelXception = None

    modelXceptionWoTop = Xception(input_tensor=input_tensor, weights='imagenet', include_top=False)

    modelXception = Sequential()

    modelXception.add(GlobalAveragePooling2D())

    modelXception.add(Dense(2, activation='softmax'))

    modelXception.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

    modelXception.fit(X_train, Y_train,
              batch_size=10, epochs=200, verbose=2,
              validation_data=(X_test, Y_test))

    K.clear_session()

# modelXceptionWoTop.summary()

# modelXception = Xception(input_tensor=input_tensor, weights='imagenet')

# modelXception.summary()

Train on 1600 samples, validate on 400 samples
Epoch 1/200
 - 3s - loss: 0.7024 - acc: 0.2850 - val_loss: 0.6984 - val_acc: 0.2725
Epoch 2/200
 - 3s - loss: 0.6953 - acc: 0.3950 - val_loss: 0.6916 - val_acc: 0.6700
Epoch 3/200
 - 3s - loss: 0.6884 - acc: 0.7006 - val_loss: 0.6852 - val_acc: 0.6850
Epoch 4/200
 - 3s - loss: 0.6817 - acc: 0.7106 - val_loss: 0.6790 - val_acc: 0.6900
Epoch 5/200
 - 3s - loss: 0.6751 - acc: 0.7106 - val_loss: 0.6730 - val_acc: 0.6800
Epoch 6/200
 - 3s - loss: 0.6689 - acc: 0.7081 - val_loss: 0.6672 - val_acc: 0.6825
Epoch 7/200
 - 3s - loss: 0.6628 - acc: 0.7094 - val_loss: 0.6616 - val_acc: 0.6800
Epoch 8/200
 - 3s - loss: 0.6570 - acc: 0.7081 - val_loss: 0.6561 - val_acc: 0.6850
Epoch 9/200
 - 3s - loss: 0.6513 - acc: 0.7081 - val_loss: 0.6509 - val_acc: 0.6800
Epoch 10/200
 - 3s - loss: 0.6459 - acc: 0.7094 - val_loss: 0.6458 - val_acc: 0.6825
Epoch 11/200
 - 3s - loss: 0.6406 - acc: 0.7113 - val_loss: 0.6408 - val_acc: 0.6825
Epoch 12/200
 - 3s - loss: 

In [0]:
from keras.applications import VGG16
from keras import backend as K

for i in range(100):
    
    input_tensor = Input(shape=(224, 224, 3))

    modelVGG16WoTop = None
    modelVGG16 = None

    modelVGG16WoTop = VGG16(input_tensor=input_tensor, weights='imagenet', include_top=False)

    modelVGG16 = Sequential()

    modelVGG16.add(modelVGG16WoTop)
    modelVGG16.add(Flatten())
    modelVGG16.add(Dense(4096, activation='relu'))
    modelVGG16.add(Dropout(0.5))
    modelVGG16.add(Dense(4096, activation='relu'))
    modelVGG16.add(Dropout(0.5))

    modelVGG16.add(Dense(2, activation='softmax'))

    modelVGG16.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    modelVGG16.fit(X_train, Y_train,
              batch_size=2, epochs=8, verbose=1,
              validation_data=(X_test, Y_test))
    
    K.clear_session()

Train on 1600 samples, validate on 400 samples
Epoch 1/8
1600/1600 [==============================] - 183s 114ms/step - loss: 7.9947 - acc: 0.5000 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 2/8
 648/1600 [===========>..................] - ETA: 1:41 - loss: 8.1883 - acc: 0.4892

KeyboardInterrupt: ignored

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions

modelResNet50WoTop = None
modelResNet50 = None

input_tensor = Input(shape=(224, 224, 3))
modelResNet50WoTop = ResNet50(input_tensor=input_tensor, weights='imagenet', include_top=False)

modelResNet50 = Sequential()

modelResNet50.add(modelResNet50WoTop)
modelResNet50.add(Flatten())
modelResNet50.add(Dense(2, activation='softmax'))

modelResNet50.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

modelResNet50.fit(X_train, Y_train,
          batch_size=4, epochs=8, verbose=1,
          validation_data=(X_test, Y_test))

In [0]:
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import datetime

now = datetime.datetime.now()
modelResNet50.save('gdrive/My Drive/Colab/Model/ResNet50 {}.h5'.format(now.strftime('%m-%d %H:%M:%S')))
# modelResNet50_ = load_model('gdrive/My Drive/Colab/Model/ResNet50.h5')

# Y_pred = modelResNet50_.predict(X_test)
# print(confusion_matrix(np.argmax(Y_pred, axis = 1), np.argmax(Y_test, axis = 1)))

In [0]:
from keras import backend as K
K.clear_session()

In [0]:
from keras.applications import VGG19

input_tensor = Input(shape=(224, 224, 3))

modelVGG19WoTop = None
modelVGG19 = None

modelVGG19WoTop = VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)

modelVGG19 = Sequential()

modelVGG19.add(modelVGG19WoTop)
modelVGG19.add(Flatten())
modelVGG19.add(Dense(4096, activation='relu'))
modelVGG19.add(Dropout(0.5))
modelVGG19.add(Dense(4096, activation='relu'))
modelVGG19.add(Dropout(0.5))
modelVGG19.add(Dense(2, activation='softmax'))

modelVGG19.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

modelVGG19.fit(X_train, Y_train,
          batch_size=2, epochs=8, verbose=1,
          validation_data=(X_test, Y_test))

Train on 1600 samples, validate on 400 samples
Epoch 1/8
1600/1600 [==============================] - 285s 178ms/step - loss: 8.0040 - acc: 0.5000 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 2/8
1600/1600 [==============================] - 279s 174ms/step - loss: 8.0151 - acc: 0.5000 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 3/8
1600/1600 [==============================] - 279s 174ms/step - loss: 8.0151 - acc: 0.5000 - val_loss: 8.0151 - val_acc: 0.5000
Epoch 4/8
1338/1600 [========================>.....] - ETA: 43s - loss: 8.0630 - acc: 0.4970

KeyboardInterrupt: ignored

In [0]:
# for i in range(1000, 1010):
#     plt.figure()
#     plt.imshow(X_train[i, :, :, 0]);
#     plt.plot()

